In [ ]:
! pip install kaggle


Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


In [ ]:
from google.colab import drive
drive.mount('/content/drive')

! cp /content/drive/MyDrive/kaggle.json ~/
! mkdir ~/.kaggle
! mv ~/kaggle.json ~/.kaggle/kaggle.json
! chmod 600 ~/.kaggle/kaggle.json

Mounted at /content/drive


In [ ]:
! kaggle datasets download -d eldarsultanow/ml-bsd-dataset-integers

 98% 269M/274M [00:02<00:00, 120MB/s]
100% 274M/274M [00:02<00:00, 97.9MB/s]


In [ ]:
! unzip ml-bsd-dataset-integers.zip

Archive:  ml-bsd-dataset-integers.zip
  inflating: rank-vs-ap-integers.csv  


In [ ]:
import pandas as pd
import numpy as np
import sympy
from scipy.interpolate import interp1d
from sklearn.cluster import KMeans
from sklearn.model_selection import GridSearchCV, train_test_split
from sklearn.metrics import mean_squared_error
from keras.models import Sequential
from sklearn.preprocessing import StandardScaler
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Dropout, Conv1D, MaxPooling1D, Flatten
from sklearn.ensemble import RandomForestClassifier, GradientBoostingClassifier
from tensorflow.keras.wrappers.scikit_learn import KerasRegressor
from keras.optimizers import SGD
from keras.callbacks import EarlyStopping
from sympy import prime, isprime, primerange
import random
import math
from sklearn.metrics import mean_absolute_error
from sklearn.metrics import mean_squared_error, r2_score, accuracy_score, f1_score


In [ ]:
df = pd.read_csv("rank-vs-ap-integers.csv", index_col=False, header=None)


In [ ]:
new_columns = ['rank', 'con', 'root'] + [f'ap_{i}' for i in range(3, 1232)]

df.columns = new_columns

In [ ]:
df['rank'].value_counts()

1    62710
0    42290
2    30000
3    13616
4     1335
5       48
6        1
Name: rank, dtype: int64

In [ ]:
df_2 = df.loc[df['rank'] == 2].sample(n=13616, random_state=1)
df_3 = df.loc[df['rank'] == 3]
df_3_2 = pd.concat([df_3, df_2])
df_1 = df.loc[df['rank'] == 1].sample(n=42290, random_state=1)
df_0 = df.loc[df['rank'] == 0]
df_0_1 = pd.concat([df_1, df_0])

df_sub = pd.concat([df_0_1, df_3_2])
df_sub['rank'].value_counts()

1    42290
0    42290
3    13616
2    13616
Name: rank, dtype: int64

In [ ]:
df_sub = df_sub[~df_sub['rank'].isin([4,5,6])]

In [ ]:
df_sub['rank'].value_counts()

1    42290
0    42290
3    13616
2    13616
Name: rank, dtype: int64

In [ ]:
print(df_sub)

        rank         con  root      ap_3      ap_4      ap_5      ap_6  \
76492      1  7900158977    -1 -1.000000  1.000000 -0.063251  4.000000   
62660      1  7160450273    -1 -1.000000 -0.045161 -0.063251  2.000000   
26131      1  7368135787    -1 -0.031522 -0.045161  4.000000  1.000000   
74087      1  7601926747    -1 -1.000000 -2.000000  2.000000  2.000000   
8084       1  7716687347    -1  2.000000  1.000000  2.000000 -1.000000   
...      ...         ...   ...       ...       ...       ...       ...   
117731     2  7959469501     1 -1.000000  2.000000  1.000000  4.000000   
133602     2  7068244081     1 -1.000000  1.000000 -1.000000 -4.000000   
126113     2  7289095621     1 -1.000000 -1.000000 -1.000000 -4.000000   
113982     2  7299096653     1 -1.000000  2.000000 -0.063251  1.000000   
120408     2  7745847787     1 -2.000000 -0.045161  2.000000 -0.068315   

            ap_7  ap_8      ap_9  ...  ap_1222  ap_1223  ap_1224  ap_1225  \
76492  -4.000000   5.0 -2.000000  

In [ ]:
from sympy import primepi

def get_r_sequence(N:int):
  pi_N = primepi(N)
  res = [None]*pi_N

  for i in range(1,pi_N+1):
    res[i-1] = -1 + i*2/pi_N
  return res

print(len(get_r_sequence(2000)))
print(get_r_sequence(2000))


303
[-301/303, -299/303, -99/101, -295/303, -293/303, -97/101, -289/303, -287/303, -95/101, -283/303, -281/303, -93/101, -277/303, -275/303, -91/101, -271/303, -269/303, -89/101, -265/303, -263/303, -87/101, -259/303, -257/303, -85/101, -253/303, -251/303, -83/101, -247/303, -245/303, -81/101, -241/303, -239/303, -79/101, -235/303, -233/303, -77/101, -229/303, -227/303, -75/101, -223/303, -221/303, -73/101, -217/303, -215/303, -71/101, -211/303, -209/303, -69/101, -205/303, -203/303, -67/101, -199/303, -197/303, -65/101, -193/303, -191/303, -63/101, -187/303, -185/303, -61/101, -181/303, -179/303, -59/101, -175/303, -173/303, -57/101, -169/303, -167/303, -55/101, -163/303, -161/303, -53/101, -157/303, -155/303, -51/101, -151/303, -149/303, -49/101, -145/303, -143/303, -47/101, -139/303, -137/303, -45/101, -133/303, -131/303, -43/101, -127/303, -125/303, -41/101, -121/303, -119/303, -39/101, -115/303, -113/303, -37/101, -109/303, -107/303, -35/101, -103/303, -1/3, -33/101, -97/303, -95/

In [ ]:
primes = [-301/303, -299/303, -99/101, -295/303, -293/303, -97/101, -289/303, -287/303, -95/101, -283/303, -281/303, -93/101, -277/303, -275/303, -91/101, -271/303, -269/303, -89/101, -265/303, -263/303, -87/101, -259/303, -257/303, -85/101, -253/303, -251/303, -83/101, -247/303, -245/303, -81/101, -241/303, -239/303, -79/101, -235/303, -233/303, -77/101, -229/303, -227/303, -75/101, -223/303, -221/303, -73/101, -217/303, -215/303, -71/101, -211/303, -209/303, -69/101, -205/303, -203/303, -67/101, -199/303, -197/303, -65/101, -193/303, -191/303, -63/101, -187/303, -185/303, -61/101, -181/303, -179/303, -59/101, -175/303, -173/303, -57/101, -169/303, -167/303, -55/101, -163/303, -161/303, -53/101, -157/303, -155/303, -51/101, -151/303, -149/303, -49/101, -145/303, -143/303, -47/101, -139/303, -137/303, -45/101, -133/303, -131/303, -43/101, -127/303, -125/303, -41/101, -121/303, -119/303, -39/101, -115/303, -113/303, -37/101, -109/303, -107/303, -35/101, -103/303, -1/3, -33/101, -97/303, -95/303, -31/101, -91/303, -89/303, -29/101, -85/303, -83/303, -27/101, -79/303, -77/303, -25/101, -73/303, -71/303, -23/101, -67/303, -65/303, -21/101, -61/303, -59/303, -19/101, -55/303, -53/303, -17/101, -49/303, -47/303, -15/101, -43/303, -41/303, -13/101, -37/303, -35/303, -11/101, -31/303, -29/303, -9/101, -25/303, -23/303, -7/101, -19/303, -17/303, -5/101, -13/303, -11/303, -3/101, -7/303, -5/303, -1/101, -1/303, 1/303, 1/101, 5/303, 7/303, 3/101, 11/303, 13/303, 5/101, 17/303, 19/303, 7/101, 23/303, 25/303, 9/101, 29/303, 31/303, 11/101, 35/303, 37/303, 13/101, 41/303, 43/303, 15/101, 47/303, 49/303, 17/101, 53/303, 55/303, 19/101, 59/303, 61/303, 21/101, 65/303, 67/303, 23/101, 71/303, 73/303, 25/101, 77/303, 79/303, 27/101, 83/303, 85/303, 29/101, 89/303, 91/303, 31/101, 95/303, 97/303, 33/101, 1/3, 103/303, 35/101, 107/303, 109/303, 37/101, 113/303, 115/303, 39/101, 119/303, 121/303, 41/101, 125/303, 127/303, 43/101, 131/303, 133/303, 45/101, 137/303, 139/303, 47/101, 143/303, 145/303, 49/101, 149/303, 151/303, 51/101, 155/303, 157/303, 53/101, 161/303, 163/303, 55/101, 167/303, 169/303, 57/101, 173/303, 175/303, 59/101, 179/303, 181/303, 61/101, 185/303, 187/303, 63/101, 191/303, 193/303, 65/101, 197/303, 199/303, 67/101, 203/303, 205/303, 69/101, 209/303, 211/303, 71/101, 215/303, 217/303, 73/101, 221/303, 223/303, 75/101, 227/303, 229/303, 77/101, 233/303, 235/303, 79/101, 239/303, 241/303, 81/101, 245/303, 247/303, 83/101, 251/303, 253/303, 85/101, 257/303, 259/303, 87/101, 263/303, 265/303, 89/101, 269/303, 271/303, 91/101, 275/303, 277/303, 93/101, 281/303, 283/303, 95/101, 287/303, 289/303, 97/101, 293/303, 295/303, 99/101, 299/303, 301/303, 1]


In [ ]:
abs_prime = [abs(prime) for prime in primes]

In [ ]:
ap_cols = [col for col in df_sub.columns if col.startswith("ap_")]
df_sub[ap_cols] = df_sub[ap_cols].replace(0, np.nan)
df_sub[ap_cols] = df_sub[ap_cols].fillna(df_sub[ap_cols].mean())

In [ ]:
for i in range(len(abs_prime)):
    ap_col = f"ap_{i+3}"
    df_sub[ap_col].dropna(inplace=True)
    if (df_sub[ap_col] == 0).any():
        df_sub[ap_col].replace(0, df_sub[ap_col].mean(), inplace=True)
    df_sub[ap_col] = df_sub[ap_col].apply(lambda x, p=abs_prime[i]: x / math.sqrt(p))


In [ ]:
modified_ap = pd.concat([df_sub['rank'], df_sub.iloc[:,3:301]], axis=1)

In [ ]:
print(modified_ap)

        rank  root      ap_3      ap_4      ap_5      ap_6      ap_7  \
76492      1    -1 -1.006678  1.013446 -0.064536  4.109032 -4.137225   
62660      1    -1 -1.006678 -0.045768 -0.064536  2.054516  6.205837   
26131      1    -1 -0.031733 -0.045768  4.081220  1.027258 -0.072978   
74087      1    -1 -1.006678 -2.026891  2.040610  2.054516  4.137225   
8084       1    -1  2.013356  1.013446  2.040610 -1.027258  4.137225   
...      ...   ...       ...       ...       ...       ...       ...   
117731     2     1 -1.006678  2.026891  1.020305  4.109032 -2.068612   
133602     2     1 -1.006678  1.013446 -1.020305 -4.109032  4.137225   
126113     2     1 -1.006678 -1.013446 -1.020305 -4.109032 -4.137225   
113982     2     1 -1.006678  2.026891 -0.064536  1.027258 -4.137225   
120408     2     1 -2.013356 -0.045768  2.040610 -0.070177  3.102919   

            ap_8      ap_9     ap_10  ...     ap_291     ap_292     ap_293  \
76492   5.207259 -2.097394  2.112087  ...  67.179218 -18.

In [ ]:
modified_ap.to_csv('final_ap.csv',index=False)

In [ ]:
df = pd.read_csv('final_ap.csv')

In [ ]:
df = df.sample(frac=0.2, random_state=42)

In [ ]:
feature_col = [col for col in df.columns if col.startswith('ap_')]
X = df[feature_col]
y = df['rank']
n_clusters = 4
kmeans = KMeans(n_clusters=n_clusters)
kmeans.fit(X)
df['cluster_label'] = kmeans.labels_

/usr/local/lib/python3.9/dist-packages/sklearn/cluster/_kmeans.py:870: FutureWarning: The default value of `n_init` will change from 10 to 'auto' in 1.4. Set the value of `n_init` explicitly to suppress the warning
  warnings.warn(


In [ ]:
print(df['cluster_label'])

14521    0
12772    3
64844    1
50963    3
16822    3
        ..
92088    1
18005    3
16357    0
67044    3
75465    1
Name: cluster_label, Length: 22362, dtype: int32


In [ ]:
X_reduced = df[['cluster_label']+feature_col].values
y_reduced = df['rank'].values

In [ ]:
print(X_reduced)

In [ ]:
X_reduced_np = np.array(X_reduced)
y_reduced_np =np.array(y_reduced) 

In [ ]:
X_train, X_test, y_train, y_test = train_test_split(X_reduced_np, y_reduced_np, test_size=0.4, random_state=42)


In [ ]:
scaler = StandardScaler()
X_train = scaler.fit_transform(X_train)
X_test = scaler.transform(X_test)

In [ ]:
def create_model(filters=64, kernel_size=3, dropout=0.5):
    model = Sequential()
    model.add(Conv1D(filters=filters, kernel_size=kernel_size, activation='relu', input_shape=(X_train.shape[1], 1)))
    model.add(MaxPooling1D(pool_size=2))
    model.add(Flatten())
    model.add(Dense(128, activation='relu'))
    model.add(Dropout(dropout))
    model.add(Dense(1))
    model.compile(loss='mse', optimizer='adam')
    return model

In [ ]:
model = create_model(filters=64, kernel_size=3, dropout=0.5)


In [ ]:
model.fit(X_train.reshape((X_train.shape[0], X_train.shape[1], 1)), y_train, epochs=10, batch_size=32, verbose=0)

In [ ]:
sgd = SGD(lr=0.01, momentum=0.9)

model.compile(optimizer=sgd,
              loss='mse',
              metrics=['mae', 'mse'])

history = model.fit(X_train, y_train, epochs=50, batch_size=64, validation_data=(X_test, y_test), verbose=1)


y_pred = model.predict(X_test)

In [ ]:
mse, mae, accuracy = model.evaluate(X_test.reshape((X_test.shape[0], X_test.shape[1], 1)), y_test, verbose=0)

print("Accuracy:", accuracy)

Accuracy: 0.999197781085968
